<a href="https://colab.research.google.com/github/arjunxpratap/OCR-Receipt-using-Hugging-Face-API/blob/main/OCR_Receipt_using_Hugging_Face_Api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install torch
!pip install pillow
!pip install opencv-python


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [3]:
import os
from PIL import Image
import requests
import torch
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import re


In [4]:
# Load the Hugging Face TrOCR model and processor
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-printed')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-printed')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
# Path to the image you uploaded in Colab
image_path = '/content/1013-receipt.jpg'
image = Image.open(image_path)

# Resize the image to improve OCR performance
image = image.resize((2048, 2048))

# Display the image (optional)
image.show()

# Preprocess the image using the processor
pixel_values = processor(images=image, return_tensors="pt").pixel_values


In [6]:
# Generate text from the image
generated_ids = model.generate(pixel_values, max_length=50)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("Extracted Text:", generated_text)


Extracted Text: :


In [7]:
# Function to extract fields from the generated text
def extract_fields(text):
    invoice_number = re.search(r'Invoice Number[:\s]*([A-Za-z0-9\-]+)', text)
    date = re.search(r'Date[:\s]*(\d{2}[-/]\d{2}[-/]\d{4})', text)
    total_amount = re.search(r'Total[:\s]*([\d,.]+)', text)

    return {
        "Invoice Number": invoice_number.group(1) if invoice_number else None,
        "Date": date.group(1) if date else None,
        "Total Amount": total_amount.group(1) if total_amount else None,
    }

# Extract and print the fields
fields = extract_fields(generated_text)
print("Fields Extracted:", fields)


Fields Extracted: {'Invoice Number': None, 'Date': None, 'Total Amount': None}


In [8]:
from transformers import pipeline

# Load a pre-trained named entity recognition model
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")

# Apply the NER model to the extracted text
entities = ner_pipeline(generated_text)

# Print out the recognized entities
print("Recognized Entities:", entities)


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

Recognized Entities: []
